In [2]:
pip install docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx: filename=docx-0.2.4-py3-none-any.whl size=53892 sha256=bd9fa53fa773933d3f1233127a04b6401da8dd5d56c2b0d7eb5b18753319cbab
  Stored in directory: /root/.cache/pip/wheels/81/f5/1d/e09ba2c1907a43a4146d1189ae4733ca1a3bfe27ee39507767
Successfully built docx


In [4]:
!pip install --upgrade python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.3 MB/s eta 0:00:00


In [6]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.9 MB/s eta 0:00:00


In [7]:
import spacy
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize
import re
from docx import Document
from PyPDF2 import PdfReader

In [8]:
# Ensure WordNet is downloaded (for synonym generation)
nltk.download("wordnet")
nltk.download("punkt")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
# Load spaCy model for NLP processing
nlp = spacy.load("en_core_web_sm")

In [11]:
# Define a function to get synonyms using WordNet
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

In [12]:
# Function to analyze and suggest simpler words
def simplify_terms(text):
    doc = nlp(text)
    simplified_text = text

    # Iterate over tokens and replace complex words with simpler synonyms
    for token in doc:
        if token.pos_ in ['NOUN', 'VERB', 'ADJ']:  # Simplify nouns, verbs, and adjectives
            synonyms = get_synonyms(token.text)
            if len(synonyms) > 0:
                simplified_word = min(synonyms, key=len)  # Choose the simplest synonym (shortest)
                # Ensure context correctness by checking synonym similarity
                if simplified_word.lower() != token.text.lower():
                    simplified_text = simplified_text.replace(token.text, simplified_word)

    return simplified_text

In [13]:
# Function to identify key data science concepts in the manuscript
def identify_data_science_terms(text):
    # Example list of technical terms to identify and simplify
    data_science_terms = [
        "algorithm", "data frame", "machine learning", "neural network", "regression",
        "classification", "statistics", "supervised learning", "unsupervised learning",
        "overfitting", "bias", "variance", "clustering", "data set", "feature engineering",
        "predictive model", "model training", "decision tree", "k-means", "support vector machine",
        "cross-validation", "ensemble method", "deep learning", "reinforcement learning", "natural language processing",
        "convolutional neural network", "gradient descent"
    ]

    found_terms = []
    for term in data_science_terms:
        if re.search(r'\b' + re.escape(term) + r'\b', text.lower()):
            found_terms.append(term)

    return found_terms

In [14]:
# Function to read text from a DOCX file
def read_docx(file_path):
    doc = Document(file_path)
    text = "\n".join([para.text for para in doc.paragraphs])
    return text

# Function to read text from a PDF file
def read_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# Function to handle multiple input formats (Text, DOCX, PDF)
def read_manuscript(file_path):
    if file_path.endswith('.txt'):
        with open(file_path, 'r') as file:
            return file.read()
    elif file_path.endswith('.docx'):
        return read_docx(file_path)
    elif file_path.endswith('.pdf'):
        return read_pdf(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a .txt, .docx, or .pdf file.")


In [15]:

# Function to generate feedback report
def generate_feedback_report(text):
    # Identify key data science terms in the manuscript
    found_terms = identify_data_science_terms(text)

    # Generate simplified text for easier understanding
    simplified_text = simplify_terms(text)

    # Create feedback report
    feedback = {
        "identified_terms": found_terms,
        "simplified_text": simplified_text,
        "overall_feedback": "The manuscript contains complex technical terms which have been simplified for a young audience. Review the simplified text for accuracy and relevance."
    }

    return feedback

In [17]:
# Function to extract key sentences with complex terminology
def extract_complex_sentences(text):
    complex_sentences = []
    sentences = sent_tokenize(text)

    # Identify sentences that contain data science terms
    for sentence in sentences:
        if any(term.lower() in sentence.lower() for term in ["algorithm", "neural network", "overfitting", "model", "machine learning"]):
            complex_sentences.append(sentence)

    return complex_sentences

In [18]:
# Function to suggest more relatable examples for technical terms
def suggest_examples_for_terms(text):
    term_examples = {
        "neural network": "Think of it as a brain trying to recognize objects or patterns from a picture, just like how humans learn.",
        "overfitting": "Imagine a teacher who teaches only one specific question over and over again. The student might do well on that question but fail others.",
        "classification": "It's like sorting items into different boxes based on their characteristics, like separating animals into cats, dogs, and birds.",
        "regression": "This is like trying to predict how much someone will score on a test based on how much they studied."
    }

    for term, example in term_examples.items():
        text = text.replace(term, f"{term} (For example: {example})")

    return text

In [30]:
# Main function to process the manuscript, extract terms, simplify text, and provide feedback
def process_manuscript(file_path):
    text = read_manuscript(file_path)

    # Generate the feedback report
    feedback = generate_feedback_report(text)

    # Extract complex sentences
    complex_sentences = extract_complex_sentences(text)

    # Suggest examples for technical terms
    enhanced_text = suggest_examples_for_terms(feedback['simplified_text'])

    # Display the results
    print("Identified Data Science Terms:")
    print(feedback['identified_terms'])

    print("\nSimplified Text:")
    print(feedback['simplified_text'])

    print("\nComplex Sentences in the Manuscript:")
    for sentence in complex_sentences:
        print(f"- {sentence}")

    print("\nEnhanced Text with Examples for Technical Terms:")
    print(enhanced_text)

    print("\nOverall Feedback:")
    print(feedback['overall_feedback'])

# Example usage
if __name__ == "__main__":
    # Provide the file path to the manuscript
    file_path = "manuscript.pdf"